# 3. Modules & Losses

In [ ]:
import torch
import torch.nn as nn

## Creation and Usage

Instantiate a Linear module from the `torch.nn` library

In [ ]:
linear_regression_model = nn.Linear(in_features=5, out_features=2)  # Linear <=> Fully Connected

In [ ]:
linear_regression_model.weight  # Parameters of a module are randomly initialised

In [ ]:
type(linear_regression_model.weight)

A `Parameter` is a Tensor which is automatically added to the list of parameters when used within a model.

Check/Get all module parameters

In [ ]:
for name, tensor in linear_regression_model.named_parameters():
    print("{:6s}  -  {}".format(name, tensor.shape))

In [ ]:
list(linear_regression_model.parameters())

Calling a module on an input Tensor

In [ ]:
# Pytorch Modules operate on batches. It allows to process multiple datapoints in parallel
# All modules in torch.nn are written to produce outputs for a batch of multiple inputs at the same time.

batch_size = 3
feature_size = 5
x = torch.randn(batch_size, feature_size) # batch of 3 samples with 5 features each

print(x, "\n\n", x.shape)

In [ ]:
predicted_y = linear_regression_model(x)  # Note, you do not call explicitely forward
print(predicted_y, "\n\n", predicted_y.shape)

---

## Building Custom modules

In [ ]:
import torch.nn.functional as F

class MyNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyNeuralNetwork, self).__init__()
        
        self.linear_1 = nn.Linear(input_size, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = self.linear_2(out)
        return out

In [ ]:
model = MyNeuralNetwork(input_size=10, hidden_size=5,  num_classes=2)

In [ ]:
print(model)

In [ ]:
x = torch.rand(5, 10)   # the first dimension is reserved for the 'batch_size'
out = model(x)   # this calls model.forward(x)

print(out.shape)

---

## Composing modules with `torch.nn.Sequential`


In [ ]:
neural_net = torch.nn.Sequential(
    torch.nn.Linear(10, 5),
    torch.nn.ReLU(),
    torch.nn.Linear(5, 2),
)

print(neural_net)

In [ ]:
# Run the model:
neural_net(x)

---

## Moving your model to GPU

In [ ]:
model.cuda()  # No need to assign it

In [ ]:
device = torch.device("cuda:0")
model.to(device)  # No need to assign it

---

## Storing and loading models

### The easy way

In [ ]:
torch.save(model, "my_model.pt")

In [ ]:
my_model_loaded = torch.load("my_model.pt")

In [ ]:
print(model.linear_2.bias)
print(my_model_loaded.linear_2.bias)

### The recommended way

In [ ]:
torch.save(model.state_dict(), "my_model_state_dict.pt")

In [ ]:
my_model_loaded = MyNeuralNetwork(10, 5, 2)
my_model_loaded.load_state_dict(torch.load("my_model_state_dict.pt"))

In [ ]:
print(model.linear_2.bias)
print(my_model_loaded.linear_2.bias)

---
## Losses

PyTorch comes with a lot of predefined loss functions :
- `L1Loss`
- `MSELoss`
- `CrossEntropyLoss`
- `NLLLoss`
- `PoissonNLLLoss`
- `KLDivLoss`
- `BCELoss`
- `...`

In [ ]:
loss_function = nn.L1Loss()

In [ ]:
x = torch.Tensor([1,1,1])
y = torch.Tensor([1,2,3])

loss_function(x, y)

---
# Building our training loop (3 / 5)

In [ ]:
# INITIALIZATION

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, RandomCrop
from torchvision.datasets import ImageFolder

device = torch.device("cpu")

transform = Compose((RandomCrop((50, 50)), ToTensor()))
dataset = ImageFolder(root="../alien-vs-predator/", transform=transform)
loader = DataLoader(dataset, batch_size=5, shuffle=True)

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(7500, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 2),
)
model.to(device)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
# TRAINING LOOP

for samples, labels in loader:
    samples = samples.to(device)
    labels = labels.to(device)
    predictions = model(samples)
    loss = loss_fn(predictions, labels)
    # compute gradients
    # update model parameters